## Male CNS - BANC landmarks
In this notebook, we will generate a "shortcut" to transform data between BANC and the male CNS. The general idea is:
1. Evenly sample points in male CNS
2. Transform the VNC and the brain part via their respective JRC2018 templates to the BANC template

In [1]:
import numpy as np
import pandas as pd

import cloudvolume as cv

import flybrains
import navis

In [9]:
del flybrains.JRCFIB2022M._mesh_vnc

In [10]:
# Grab the meshes for brain and VNC
brain = flybrains.JRCFIB2022M.mesh_brain
vnc = flybrains.JRCFIB2022M.mesh_vnc

In [11]:
all_verts = np.vstack([brain.vertices, vnc.vertices])
bounds = np.vstack([all_verts.min(axis=0), all_verts.max(axis=0)])
bounds

array([[  45319.40625  ,   38146.7109375,   79817.3828125],
       [ 726917.875    ,  550810.25     , 1072833.875    ]])

In [12]:
# Add 10 microns around the mesh
offset = 10000
bounds_off = bounds.astype(int)
bounds_off[0, :] -= offset
bounds_off[1, :] += offset
bounds_off

array([[  35319,   28146,   69817],
       [ 736917,  560810, 1082833]])

In [13]:
# Sample points within the volume
res = 30000
sample = (
    np.mgrid[
        bounds_off[0, 0] : bounds_off[1, 0] : res,
        bounds_off[0, 1] : bounds_off[1, 1] : res,
        bounds_off[0, 2] : bounds_off[1, 2] : res,
    ]
    .reshape(3, -1)
    .T
)

# Drop points outside the volume
sample = sample[navis.in_volume(sample, brain) | navis.in_volume(sample, vnc)]
sample.shape

(1170, 3)

In [ ]:
# Next, we have to split the sampled points into:
# 1. the brain part which we will mirror via JRC2018M
# 2. the VNC part which we will mirror via JRCVNC2018M
is_brain = navis.in_volume(sample, brain)
is_vnc = navis.in_volume(sample, vnc)

# Remove some points in the neck
is_vnc[sample[:, 2] <= (64500 * 8)] = False

# Make sure no point is in both
(is_brain & is_vnc).sum()

np.int64(0)

In [ ]:
navis.plot3d([brain, vnc, sample[is_vnc | is_brain]], scatter_kws=dict(color="r"))

In [45]:
# Transform the points
sample_xf_brain = navis.xform_brain(sample[is_brain], source="JRCFIB2022M", target="JRC2018M")
sample_xf_brain = navis.xform_brain(sample_xf_brain, source="JRC2018M", target="JRC2018F")
sample_xf_brain = navis.xform_brain(sample_xf_brain, source="JRC2018F", target="BANC")

Transform path: JRCFIB2022M -> JRCFIB2022Mum -> JRC2018M
Transform path: JRC2018M -> JRC2018U -> JRC2018F
Transform path: JRC2018F -> BANCum -> BANC


In [46]:
# Note: we have to avoid navis trying to go via FlyWire -> JRC2018F
# We have to make it a multi-step process
sample_xf_vnc = navis.xform_brain(sample[is_vnc], source="JRCFIB2022M", target="MANC")
sample_xf_vnc = navis.xform_brain(sample_xf_vnc, source="MANC", target="JRCVNC2018M")
sample_xf_vnc = navis.xform_brain(sample_xf_vnc, source="JRCVNC2018M", target="JRCVNC2018F")
sample_xf_vnc = navis.xform_brain(sample_xf_vnc, source="JRCVNC2018F", target="BANC")

Transform path: JRCFIB2022M -> JRCFIB2022Mum -> MANCum-JRCFIB2022Mum(post) -> MANCum -> MANC
Transform path: MANC -> MANCum -> JRCVNC2018M
Transform path: JRCVNC2018M -> JRCVNC2018U -> JRCVNC2018F
Transform path: JRCVNC2018F -> BANCum -> BANC


In [ ]:
# This also looks decent
navis.plot3d([flybrains.BANC.mesh_whole_brain, sample_xf_brain, sample_xf_vnc])

In [53]:
# Bring it together
source = pd.DataFrame(np.vstack((sample[is_brain], sample[is_vnc])), columns=['x_mcns', 'y_mcns', 'z_mcns']).round().astype(int)
target = pd.DataFrame(np.vstack((sample_xf_brain, sample_xf_vnc)), columns=['x_banc', 'y_banc', 'z_banc']).round().astype(int)
lm = pd.concat((source, target), axis=1)
lm

,x_mcns,y_mcns,z_mcns,x_banc,y_banc,z_banc
0,65319,208146,279817,118834,142836,145730
1,65319,208146,309817,122311,171460,152858
2,65319,238146,279817,118903,156510,124176
3,65319,238146,309817,122146,183473,132195
4,65319,268146,279817,115745,167695,104015
...,...,...,...,...,...,...
1150,515319,448146,579817,620159,576617,178627
1151,515319,448146,609817,622200,601131,191230
1152,515319,478146,579817,630554,589034,154958
1153,515319,478146,609817,633261,611066,170265


In [54]:
# To fix stuff in the neck connective, I have also added some manual landmarks
man_banc = np.array([[126451.65625, 127035.5703125, 2932.23291015625],
 [122723.3203125, 116807.578125, 3134.591796875],
 [119250.8828125, 105523.59375, 3258.674072265625],
 [118965.1015625, 92042.8515625, 3095.716796875]]) * [4,4,45]
man_mcns = np.array([[50231.6875, 50495.99609375, 59463.44921875],
 [49946.9921875, 49975.8203125, 55865.98828125],
 [49630.0, 49596.984375, 51604.8828125],
 [49056.0078125, 46168.29296875, 47803.4921875]]) * 8

lm_extra = pd.DataFrame(
    np.hstack((man_mcns, man_banc)),
    columns=['x_mcns', 'y_mcns', 'z_mcns', 'x_banc', 'y_banc', 'z_banc']
)

lm = pd.concat((lm, lm_extra), axis=0).reset_index(drop=True).round().astype(int)
lm

,x_mcns,y_mcns,z_mcns,x_banc,y_banc,z_banc
0,65319,208146,279817,118834,142836,145730
1,65319,208146,309817,122311,171460,152858
2,65319,238146,279817,118903,156510,124176
3,65319,238146,309817,122146,183473,132195
4,65319,268146,279817,115745,167695,104015
...,...,...,...,...,...,...
1154,515319,508146,579817,648754,594926,132325
1155,401854,403968,475708,505807,508142,131950
1156,399576,399807,446928,490893,467230,141057
1157,397040,396776,412839,477004,422094,146640


In [55]:
lm.to_csv('maleCNS_BANC_landmarks_nm.csv', index=False)